# SF-DAT-21 | Unit Project 3

In this project, you will perform a logistic regression on the admissions data we've been working with in Unit Projects 1 and 2.

In [80]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pylab as pl
from sklearn import linear_model
%matplotlib inline

In [115]:
df_raw = pd.read_csv("admissions.csv")
df = df_raw.dropna()
print df.head()

   admit  gre   gpa  prestige
0      0  380  3.61         3
1      1  660  3.67         3
2      1  800  4.00         1
3      1  640  3.19         4
4      0  520  2.93         4


In [116]:
len(df)

397

## Part 1. Frequency Tables

#### Question 1. Let's create a frequency table of our variables.

In [117]:
# frequency table for prestige and whether or not someone was admitted
df.admit.value_counts()

0    271
1    126
Name: admit, dtype: int64

In [118]:
df.prestige.value_counts()

2    148
3    121
4     67
1     61
Name: prestige, dtype: int64

## Part 2. Return of dummy variables

#### Question 2.1. Create class or dummy variables for prestige.

In [119]:
dummy_ranks = pd.get_dummies(df.prestige, prefix = 'prestige')

#### Question 2.2. When modeling our class variables, how many do we need?

Answer: we need 4 class variables, one for every prestige value.

## Part 3. Hand calculating odds ratios

Develop your intuition about expected outcomes by hand calculating odds ratios.

In [120]:
cols_to_keep = ['admit', 'gre', 'gpa']
handCalc = df[cols_to_keep].join(dummy_ranks.ix[:, 'prestige_1':])
print handCalc.head()

   admit  gre   gpa  prestige_1.0  prestige_2.0  prestige_3.0  prestige_4.0
0      0  380  3.61             0             0             1             0
1      1  660  3.67             0             0             1             0
2      1  800  4.00             1             0             0             0
3      1  640  3.19             0             0             0             1
4      0  520  2.93             0             0             0             1


In [121]:
len(handCalc)

397

In [162]:
len(handCalc.prestige_1)

397

In [123]:
handCalc.columns = ['admit', 'gre', 'gpa','prestige_1','prestige_2','prestige_3','prestige_4']

In [124]:
# crosstab prestige 1 admission
# frequency table cutting prestige and whether or not someone was admitted
pd.crosstab(handCalc.prestige_1, handCalc.admit, rownames=['prestige_1'], colnames=['admission'])

admission,0,1
prestige_1,,
0,243,93
1,28,33


#### Question 3.1. Use the cross tab above to calculate the odds of being admitted to grad school if you attended a #1 ranked college.

In [125]:
# Odds = P / 1-P 

#prob_of_beign_admitted_1_prest = 33/61
#prob_of_not_beign_admitted_1_pres = 1-33/61

prob_of_beign_admitted_1_prest = 33.0/61.0
prob_of_not_beign_admitted_1_pres = 1 - (prob_of_beign_admitted_1_prest)

Odds_admitted_prest_1 = prob_of_beign_admitted_1_prest / prob_of_not_beign_admitted_1_pres
print Odds_admitted_prest_1 

1.17857142857


#### Question 3.2. Now calculate the odds of admission if you did not attend a #1 ranked college.

In [126]:
# Odds = P / 1-P 

#prob_of_beign_admitted_not_1_prest = 93/336
#prob_of_not_beign_admitted_1_pres = 1-93/336

prob_of_beign_admitted_not_1_prest = 93.0/336.0
prob_of_not_beign_admitted_not_1_prest = 1 - (prob_of_beign_admitted_not_1_prest)

Odds_addmitted_not_prest_1 = prob_of_beign_admitted_not_1_prest / prob_of_not_beign_admitted_not_1_prest
print Odds_addmitted_not_prest_1

0.382716049383


#### Question 3.3. Calculate the odds ratio.

In [127]:
Odds_admitted_prest_1/Odds_addmitted_not_prest_1

3.079493087557604

#### Question 3.4. Write this finding in a sentenance:

Answer: Thus, the odds of being admitted if attended a prestige_1 shcool is 3.08 higher compared to not attending prestige_1 school.

#### Question 3.5. Print the cross tab for prestige_4.

In [128]:
pd.crosstab(handCalc.prestige_4, handCalc.admit, rownames=['prestige_4'], colnames=['admission'])

admission,0,1
prestige_4,,
0,216,114
1,55,12


#### Question 3.6. Calculate the OR.

In [129]:
prob_of_beign_admitted_4_prest = 12/67.0
prob_of_not_beign_admitted_4_pres = 1 - (prob_of_beign_admitted_4_prest)

Odds_admitted_prest_4 = prob_of_beign_admitted_4_prest / prob_of_not_beign_admitted_4_pres
print Odds_admitted_prest_4 

0.218181818182


In [130]:
prob_of_beign_admitted_not_4_prest = 114.0/330.0
prob_of_not_beign_admitted_not_4_prest = 1 - (prob_of_beign_admitted_not_4_prest)

Odds_addmitted_not_prest_4 = prob_of_beign_admitted_not_4_prest / prob_of_not_beign_admitted_not_4_prest
print Odds_addmitted_not_prest_4

0.527777777778


In [131]:
Odds_admitted_prest_4 /Odds_addmitted_not_prest_4

0.4133971291866028

#### Question 3.7. Write this finding in a sentence.

Answer: the odds of beign admitted if attend prestige 4 school is 0.4 lower compared not attending prestige_4 school.

## Part 4. Analysis

In [132]:
# create a clean data frame for the regression
cols_to_keep = ['admit', 'gre', 'gpa']
data = df[cols_to_keep].join(dummy_ranks.ix[:, 'prestige_2':])
print data.head()

   admit  gre   gpa  prestige_2.0  prestige_3.0  prestige_4.0
0      0  380  3.61             0             1             0
1      1  660  3.67             0             1             0
2      1  800  4.00             0             0             0
3      1  640  3.19             0             0             1
4      0  520  2.93             0             0             1


We're going to add a constant term for our Logistic Regression.  The statsmodels function we're going to be using requires that intercepts/constants are specified explicitly.

In [133]:
# manually add the intercept
data['intercept'] = 1.0
data.head()

,admit,gre,gpa,prestige_2.0,prestige_3.0,prestige_4.0,intercept
0,0,380,3.61,0,1,0,1
1,1,660,3.67,0,1,0,1
2,1,800,4.00,0,0,0,1
3,1,640,3.19,0,0,1,1
4,0,520,2.93,0,0,1,1


#### Question 4.1. Set the covariates to a variable called train_cols.

In [134]:
train_cols = data[ ['gre', 'gpa', 'prestige_2.0','prestige_3.0','prestige_4.0','intercept'] ]

#### Question 4.2. Fit the model.

In [150]:
logit_mod = sm.Logit(data.admit, train_cols)
logit_res = logit_mod.fit()

Optimization terminated successfully.
         Current function value: 0.573854
         Iterations 6


#### Question 4.3. Print the summary results.

In [151]:
print logit_res.summary()

                           Logit Regression Results                           
Dep. Variable:                  admit   No. Observations:                  397
Model:                          Logit   Df Residuals:                      391
Method:                           MLE   Df Model:                            5
Date:                Sun, 27 Mar 2016   Pseudo R-squ.:                 0.08166
Time:                        01:05:49   Log-Likelihood:                -227.82
converged:                       True   LL-Null:                       -248.08
                                        LLR p-value:                 1.176e-07
                   coef    std err          z      P>|z|      [95.0% Conf. Int.]
--------------------------------------------------------------------------------
gre              0.0022      0.001      2.028      0.043      7.44e-05     0.004
gpa              0.7793      0.333      2.344      0.019         0.128     1.431
prestige_2.0    -0.6801      0.317     -2.14

#### Question 4.4. Calculate the odds ratios of the coeffincients and their 95% CI intervals

hint 1: np.exp(X)

hint 2: conf['OR'] = params

        conf.columns = ['2.5%', '97.5%', 'OR']

In [137]:
print np.exp(logit_res.params)

gre             1.002221
gpa             2.180027
prestige_2.0    0.506548
prestige_3.0    0.262192
prestige_4.0    0.211525
intercept       0.020716
dtype: float64


In [138]:
coefficients = logit_res.params
confidence_interval = logit_res.conf_int()
confidence_interval['OR'] = coefficients
confidence_interval.columns = ['2.5%', '97.5%', 'OR']
print np.exp(confidence_interval)

                  2.5%     97.5%        OR
gre           1.000074  1.004372  1.002221
gpa           1.136120  4.183113  2.180027
prestige_2.0  0.272168  0.942767  0.506548
prestige_3.0  0.133377  0.515419  0.262192
prestige_4.0  0.093329  0.479411  0.211525
intercept     0.002207  0.194440  0.020716


#### Question 4.5. Interpret the OR of Prestige_2.

Answer: Thus, the odds of being admitted if attended a prestige_2 shcool is 0.51 lower compared to not attending prestige_2 school.

#### Question 4.6. Interpret the OR of GPA.

Answer: Thus, the odds of being admitted if getting a higher gpa is 2.12 higer compared to getting lower gpa.

## Part 5: Predicted probablities


As a way of evaluating our classifier, we're going to recreate the dataset with every logical combination of input values.  This will allow us to see how the predicted probability of admission increases/decreases across different variables.  First we're going to generate the combinations using a helper function called cartesian (above).

We're going to use np.linspace to create a range of values for "gre" and "gpa".  This creates a range of linearly spaced values from a specified min and maximum value--in our case just the min/max observed values.

In [139]:
def cartesian(arrays, out=None):
    """
    Generate a cartesian product of input arrays.
    Parameters
    ----------
    arrays : list of array-like
        1-D arrays to form the cartesian product of.
    out : ndarray
        Array to place the cartesian product in.
    Returns
    -------
    out : ndarray
        2-D array of shape (M, len(arrays)) containing cartesian products
        formed of input arrays.
    Examples
    --------
    >>> cartesian(([1, 2, 3], [4, 5], [6, 7]))
    array([[1, 4, 6],
           [1, 4, 7],
           [1, 5, 6],
           [1, 5, 7],
           [2, 4, 6],
           [2, 4, 7],
           [2, 5, 6],
           [2, 5, 7],
           [3, 4, 6],
           [3, 4, 7],
           [3, 5, 6],
           [3, 5, 7]])
    """

    arrays = [np.asarray(x) for x in arrays]
    dtype = arrays[0].dtype

    n = np.prod([x.size for x in arrays])
    if out is None:
        out = np.zeros([n, len(arrays)], dtype=dtype)

    m = n / arrays[0].size
    out[:,0] = np.repeat(arrays[0], m)
    if arrays[1:]:
        cartesian(arrays[1:], out=out[0:m,1:])
        for j in xrange(1, arrays[0].size):
            out[j*m:(j+1)*m,1:] = out[0:m,1:]
    return out

In [140]:
# instead of generating all possible values of GRE and GPA, we're going
# to use an evenly spaced range of 10 values from the min to the max
gres = np.linspace(data['gre'].min(), data['gre'].max(), 10)

print gres
# array([ 220.        ,  284.44444444,  348.88888889,  413.33333333,
#         477.77777778,  542.22222222,  606.66666667,  671.11111111,
#         735.55555556,  800.        ])

gpas = np.linspace(data['gpa'].min(), data['gpa'].max(), 10)

print gpas
# array([ 2.26      ,  2.45333333,  2.64666667,  2.84      ,  3.03333333,
#         3.22666667,  3.42      ,  3.61333333,  3.80666667,  4.        ])

# enumerate all possibilities
combos = pd.DataFrame(cartesian([gres, gpas, [1, 2, 3, 4], [1.]]))

[ 220.          284.44444444  348.88888889  413.33333333  477.77777778
  542.22222222  606.66666667  671.11111111  735.55555556  800.        ]
[ 2.26        2.45333333  2.64666667  2.84        3.03333333  3.22666667
  3.42        3.61333333  3.80666667  4.        ]


#### Question 5.1. Recreate the dummy variables.

In [141]:
# recreate the dummy variables
combos.columns = ['gre', 'gpa', 'prestige', 'intercept']
dummy_ranks = pd.get_dummies(combos.prestige, prefix = 'prestige')
dummy_ranks.columns = ['prestige_1', 'prestige_2', 'prestige_3', 'prestige_4']
# keep only what we need for making predictions
cols_to_keep = ['gre', 'gpa', 'prestige', 'intercept']
combos = combos[cols_to_keep].join(dummy_ranks.ix[:, 'prestige_2':])
combos.head()

,gre,gpa,prestige,intercept,prestige_2,prestige_3,prestige_4
0,220,2.260000,1,1,0,0,0
1,220,2.260000,2,1,1,0,0
2,220,2.260000,3,1,0,1,0
3,220,2.260000,4,1,0,0,1
4,220,2.453333,1,1,0,0,0


#### Question 5.2. Make predictions on the enumerated dataset.

In [161]:
combos['admit_pred'] = logit_res.predict(combos[ ['gre', 'gpa', 'prestige_2','prestige_3','prestige_4','intercept'] ])
combos.head()

,gre,gpa,prestige,intercept,prestige_2,prestige_3,prestige_4,admit_pred
0,220,2.260000,1,1,0,0,0,0.164173
1,220,2.260000,2,1,1,0,0,0.090492
2,220,2.260000,3,1,0,1,0,0.048977
3,220,2.260000,4,1,0,0,1,0.039890
4,220,2.453333,1,1,0,0,0,0.185907


#### Question 5.3. Interpret findings for the last 4 observations.

Answer: we can see how the probability of admission increases as prestige levels increase

## Bonus

Plot the probability of being admitted into graduate school, stratified by GPA and GRE score.